# Splunk App for Data Science and Deep Learning - Notebook Template for Neo4j

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with you Splunk platform by utilizing the Splunk App for Data Science and Deep Learning (DSDL) - formerly known as the Deep Learning Toolkit for Splunk (DLTK). Find more examples and information in the app and on the [DSDL splunkbase page](https://splunkbase.splunk.com/app/4607/#/details).

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import neo4j
from neo4j import GraphDatabase
# read configuration from additional DSDL services settings
configurations = json.loads(os.environ['llm_config'])
neo4j_config = configurations['graph_db']['neo4j'][0]
neo4j_uri = neo4j_config['url']
neo4j_auth = (neo4j_config['username'], neo4j_config['password'])

# or use other ways to retrieve desired configurations
#neo4j_uri = "neo4j://neo4j:7687"
#neo4j_auth = ("neo4j", "changeme")
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("neo4j version: " + neo4j.__version__)

numpy version: 1.26.4
pandas version: 2.2.3
neo4j version: 5.28.1


In [3]:
# verify connectivity to graph database
with GraphDatabase.driver(neo4j_uri, auth=neo4j_auth) as driver:
    driver.verify_connectivity()
    print(driver)

## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DSDL Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DSDL app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter. Note that no meta data param object is retrieved here. This is only available in Stage 1 Option 2.

### Option 1 - pull data from Splunk

In [12]:
from dsdlsupport import SplunkSearch as SplunkSearch

In [13]:
search = SplunkSearch.SplunkSearch()

In [14]:
df = search.as_df()
df

""


### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file or the param object. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

| inputlookup firewall_traffic.csv where used_by_malware="yes"<br>
| stats count sum(bytes_*) as sum_bytes_* sum(packets_*) as sum_packets_* by src_ip dst_ip<br>
| fit MLTKContainer mode=stage algo=barebone_neo4j * into app:neo4j_firewall_traffic

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [20]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [21]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("neo4j_firewall_traffic")

In [22]:
df

,src_ip,dst_ip,count,sum_bytes_sent,sum_bytes_received,sum_packets_received,sum_packets_sent
0,0.110.36.24,213.177.25.241,5,443,783,5,5
1,0.133.99.6,12.205.103.181,5,470,450,5,5
2,0.133.99.6,186.220.194.92,2,188,184,2,2
3,0.133.99.6,46.104.83.117,2,188,184,2,2
4,0.169.156.183,136.122.35.66,1,549,602,5,5
...,...,...,...,...,...,...,...
5619,99.74.113.249,42.240.53.15,2,188,184,2,2
5620,99.74.113.249,72.109.241.74,1,94,90,1,1
5621,99.84.226.3,73.147.88.91,9,1298,3125,21,21
5622,99.86.43.160,190.218.63.77,1,94,94,1,1


In [56]:
for _, row in df.iterrows():
    src_ip = row['src_ip']
    dst_ip = row['dst_ip']
    count = row['count']
    sum_bytes_sent = row['sum_bytes_sent']
    sum_bytes_received = row['sum_bytes_received']
    sum_packets_received = row['sum_packets_received']
    sum_packets_sent = row['sum_packets_sent']
    cypher_query = f"""
        MERGE (src:ip_address {{ip: '{src_ip}'}})
        MERGE (dst:ip_address {{ip: '{dst_ip}'}})
        CREATE (src)-[conn:connection {{
          count: {count},
          sum_bytes_sent: {sum_bytes_sent},
          sum_bytes_received: {sum_bytes_received},
          sum_packets_received: {sum_packets_received},
          sum_packets_sent: {sum_packets_sent}
        }}]->(dst)
        """
    print(cypher_query)
    break


        MERGE (src:ip_address {ip: '0.110.36.24'})
        MERGE (dst:ip_address {ip: '213.177.25.241'})
        CREATE (src)-[conn:connection {
          count: 5,
          sum_bytes_sent: 443,
          sum_bytes_received: 783,
          sum_packets_received: 5,
          sum_packets_sent: 5
        }]->(dst)
        


In [18]:
param

{'options': {'params': {'mode': 'stage', 'algo': 'barebone_template'},
  'args': ['_time', 'feature_*', 'i'],
  'feature_variables': ['_time', 'feature_*', 'i'],
  'model_name': 'barebone_template',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '600',
   'max_inputs': '100000',
   'max_memory_usage_mb': '4000',
   'max_model_size_mb': '30',
   'max_score_time': '600',
   'use_sampling': 'true'},
  'kfold_cv': None},
 'feature_variables': ['_time', 'feature_1', 'feature_2', 'feature_0', 'i']}

## Stage 2 - create and initialize a model

In [68]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['neo4j'] = GraphDatabase.driver(neo4j_uri, auth=neo4j_auth)    
    return model

# define any helper functions for neo4j
def create_connection(tx, row):
    # Create a new connection for each row in the dataframe
    cypher_query = f"""
        MERGE (src:ip_address {{ip: '{row['src_ip']}'}})
        MERGE (dst:ip_address {{ip: '{row['dst_ip']}'}})
        CREATE (src)-[conn:connection {{
          count: {row['count']},
          sum_bytes_sent: {row['sum_bytes_sent']},
          sum_bytes_received: {row['sum_bytes_received']},
          sum_packets_received: {row['sum_packets_received']},
          sum_packets_sent: {row['sum_packets_sent']}
        }}]->(dst)
        """
    result = tx.run(cypher_query)


In [69]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'neo4j': <neo4j._sync.driver.Neo4jDriver object at 0xffff5ee4b250>}


## Stage 3 - fit the model

In [62]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    with model['neo4j'].session(database="neo4j") as session:
        for _, row in df.iterrows():
            org_id = session.execute_write(create_connection, row)
            #print(f"Added: {row}")
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [66]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    records, summary, keys = model['neo4j'].execute_query(
        "MATCH (src:ip_address)-[conn:connection]->(dst:ip_address)RETURN count(conn)",
        database_="neo4j",
    )
    result = pd.DataFrame(records[0], columns=['result'])
    return result

In [67]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

   result
0    5625


## Stage 5 - save the model

In [50]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    return model

## Stage 6 - load the model

In [51]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = init(None,None)
    return model

## Stage 7 - provide a summary of the model

In [52]:
# return a model summary
def summary(model=None):
    returns = {"version": {"neo4j": neo4j.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code